##### 이미지 데이터 분류
- 데이터 : sklearn.datasets mnist => 디지털 숫자 0~9 손글씨 데이터셋
- 피처 : 이미지 픽셀 수
- 타깃 : 0 ~ 9
- 모델 : 분류 모델

1. Load Data

In [3]:
# 0. Load Module & Data
# from sklearn.datasets import fetch_openml
# mnist = fetch_openml('mnist_784', parser='auto')
# mnist.data

# # 1. 저장 시켜놓기 to_csv
import pandas as pd

# mnist_data = pd.DataFrame(mnist.data)
# mnist_data.to_csv('mnist_data.csv', index=False)
# mnist_target = pd.DataFrame(mnist.target)
# mnist_target.to_csv('mnist_target.csv', index=False)

2. 데이터 전처리
- 이미지 픽셀값 0~1로 정규화

In [4]:
mnist_data = pd.read_csv('mnist_data.csv')
feature = mnist_data / 255.0    # 0~255 이므로 0~1로 정규화
target = pd.read_csv('mnist_target.csv')

# object 변환

3. 기계학습용 데이터셋 준비
- 학습용, 검증용, 테스트용 3개 데이터셋 준비

In [5]:
# 1. 모듈 로딩
from sklearn.model_selection import train_test_split

# 2. 데이터 분리
X_train, X_test, y_train, y_test = feature[:6000], feature[6000:], target[:6000], target[6000:]

# 3. 검증용 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

print(f'X train shape : {X_train.shape}, X val shape : {X_val.shape}, X test shape : {X_test.shape}')

X train shape : (4200, 784), X val shape : (1800, 784), X test shape : (64000, 784)


4. 학습 진행  
4-1. 분류 방법 : LogisticRegression

In [6]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=10000)
lr_model.fit(X_train, y_train)

c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=10000)

In [7]:
train_score = lr_model.score(X_train, y_train)
val_score = lr_model.score(X_val, y_val)
test_score = lr_model.score(X_test, y_test)
print(f'train_score : {train_score}, val_score :{val_score}, test_score : {test_score}')

train_score : 0.9921428571428571, val_score :0.9005555555555556, test_score : 0.889734375


In [8]:
# Check parameter
print('n_iter :', lr_model.n_iter_) # 298

n_iter : [298]


4-2. 확률적경사하강법(SGD) 기반 학습  
- 대량의 데이터를 학습 시 랜덤으로 데이터셋 하나 추출, 학습 진행 -> 최적화
- 장점 : 속도 빨라짐
- 단점 : 정확도 떨어짐
- 개선 : 일정 데이터를 묶어 학습 후 최적화 진행 -> 미니배치

In [9]:
from sklearn.linear_model import SGDClassifier

In [10]:
lr_sgd = SGDClassifier(loss='log_loss')
lr_sgd.fit(X_train, y_train)

c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(loss='log_loss')

In [15]:
# Check parameter
print('n_iter :', lr_sgd.n_iter_)

# Evaluation
print(f'train score : {lr_sgd.score(X_train, y_train):.5f}, val score : {lr_sgd.score(X_val, y_val):.5f}, test score : {lr_sgd.score(X_test, y_test):.5f}')

n_iter : 130
train score : 0.98095, val score : 0.87778, test score : 0.86606


In [16]:
# Check W and b
print(f'[BEFORE] lr_sgd.coef_ :\n{lr_sgd.coef_}')
print(f'[BEFORE] lr_sgd.intercept_ :\n{lr_sgd.intercept_}')


[BEFORE] lr_sgd.coef_ :
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[BEFORE] lr_sgd.intercept_ :
[-45.02032334 -10.17220035  -8.45440182 -19.1803352   -3.88481268
   0.82936066 -26.73080473  -4.39605013 -47.39674051  -9.78058641]


In [18]:
# 추가된 데이터에 대한 추가 학습 진행 => 기존 학습으로 최적화된 상태 유지
import numpy as np

lr_sgd.partial_fit(X_val, y_val, np.array(y_val))


c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(loss='log_loss')

5. 학습 모델 성능 평가

In [11]:
# 분류 모델 성능 평가
# from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

# classification_report : 한 번에 다 출력!
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [12]:
# 클래스 비율 확인
# : 클래스가 균등해야 accuracy 가능
y_train.value_counts(), y_test.value_counts()

(class
 1        481
 4        442
 7        441
 6        434
 0        427
 9        425
 3        417
 2        392
 8        379
 5        362
 Name: count, dtype: int64,
 class
 1        7206
 7        6642
 3        6533
 2        6409
 9        6357
 0        6311
 8        6274
 6        6268
 4        6201
 5        5799
 Name: count, dtype: int64)